<a href="https://colab.research.google.com/github/kyt4n/Malaysia-COVID-19-Dataset/blob/main/covid_dataset_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Update Case Data

In [1]:
!pip install --upgrade -q gspread 

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [1]:
import json
import tweepy
import pandas as pd
import re

with open("twitter_creds.json", "r") as file:
  creds = json.load(file)

auth = tweepy.OAuthHandler(creds['api_key'], creds['api_sec'])
auth.set_access_token(creds['access_token'], creds['access_sec'])
api = tweepy.API(auth, wait_on_rate_limit=True)

username = '@DGHisham'
count = 300
tweet_id = '1416349238192009223'

tweets = tweepy.Cursor(api.user_timeline,id=username, tweet_mode="extended", since_id = tweet_id,
                       include_rts=False, exclude_replies=True).items(count)
tweets_lst = [[tweet.created_at, tweet.full_text.replace('\n', ' '), tweet.id] for tweet in tweets]

cols = "date text tweet_id".split()
tweets_df = pd.DataFrame(tweets_lst, columns=cols)

In [2]:
tweets_df.head(4).append(tweets_df.tail(4))

,date,text,tweet_id
0,2021-07-19 11:13:17,Kenyataan Akhbar KPK 19 Julai 2021 – Situasi S...,1417080095231991808
1,2021-07-19 10:36:28,"COVID-19: 19 Julai 2021 J kes sembuh:6,439 J ...",1417070830656262147
2,2021-07-19 10:10:26,"A total of 245,983 healthcare workers have bee...",1417064280847917058
3,2021-07-19 09:05:30,"10,972 kes hari ini (19 Julai 2021): Kategori ...",1417047939445231620
12,2021-07-18 07:10:14,Sehingga 18 Julai 2021: Jumlah kes COVID-19 ya...,1416656541927677953
13,2021-07-17 14:01:43,"Terkini 17Julai 2021, dikongsikan status progr...",1416397705740316681
14,2021-07-17 13:52:29,Kadar kebolehjangkitan Covid-19 atau Ro/Rt pad...,1416395385828827141
15,2021-07-17 11:26:10,Kenyataan Akhbar KPK 17 Julai 2021 – Situasi S...,1416358561169485831


In [3]:
bool_rel_idx = tweets_df.text.str.contains('terkini covid|^COVID-19[:;]', regex=True, case=False)
bool_nonrel_idx = tweets_df.text.str.contains('negeri|kluster|vaccin|vaksin', regex=True, case=False)
tweets_df = tweets_df[bool_rel_idx]
tweets_df = tweets_df.loc[~bool_nonrel_idx, :]    # without copying   # ~ invert boolean, subset exclude [non relevant rows idx] [false]

tweets_df = tweets_df[::-1].reset_index(drop=True)     # reorder rows  # reset index
#tweets_df['date'] = pd.to_datetime(tweets_df['date']).dt.date
tweets_df['date'] = pd.to_datetime(tweets_df['date'].dt.date)
tweets_df[['text','link']] = tweets_df['text'].str.split('https://', expand=True)

In [4]:
# tweets_df['text'].iloc[0]

In [5]:
tweets_df

,date,text,tweet_id,link
0,2021-07-18,"COVID-19: 18 Julai 2021 J kes sembuh: 5,778 J...",1416708430773899268,t.co/fGSlNpO4b4
1,2021-07-19,"COVID-19: 19 Julai 2021 J kes sembuh:6,439 J ...",1417070830656262147,t.co/HSEnf9pkm8


In [6]:
lst_df = []

for i in range(len(tweets_df)):
  text = tweets_df.loc[i,'text']
  
  lst_tweet = re.findall(r"[0-9]+[,.]?[0-9]*", text)
  lst_col0_8 = lst_tweet[3:12]        # elemnt 4 until 12  # total 9  
  lst_col17_20 = lst_tweet[-4:]

  importcase = re.search(r"(import)(.*?)(J)", text, re.IGNORECASE).group(0)                     # we only know something is in it
  import_total = re.search(r"[0-9]+", importcase).group(0)                                      # get the first number  # total 
  if import_total != '0':                                                                       # if not 0 
    import_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", importcase, re.IGNORECASE)
    import_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", importcase, re.IGNORECASE)
    if import_wn_re is not None:
      import_wn = import_wn_re.group(1)
      if import_bwn_re is None:
        import_bwn = str(int(import_total) - int(import_wn))
    if import_bwn_re is not None:
      import_bwn = import_bwn_re.group(1)
      if import_wn_re is None:
        import_wn = str(int(import_total) - int(import_bwn)) 
    if import_wn_re == import_bwn_re == None:
      import_wn = import_bwn = '' 
  else: import_total = import_wn = import_bwn = '0'


  maticase = re.search(r"(mati|fataliti)(.*?)(J)", text, re.IGNORECASE).group(0)                 # we only know something is in it
  mati_total = re.search(r"[0-9]+", maticase).group(0)                                           # get the first number  # total 
  if import_total != '0':                                                                        # if not 0 
    mati_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", maticase, re.IGNORECASE)
    mati_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", maticase, re.IGNORECASE)
    if mati_wn_re is not None:
      mati_wn = mati_wn_re.group(1)
      if mati_bwn_re is None:
        mati_bwn = str(int(mati_total) - int(mati_wn))
    if mati_bwn_re is not None:
      mati_bwn = mati_bwn_re.group(1)
      if mati_wn_re is None:
        mati_wn = str(int(mati_total) - int(mati_bwn)) 
    if mati_wn_re == mati_bwn_re == None:
      mati_wn = mati_bwn = '' 
  else: mati_total = mati_wn = mati_bwn = '0'

  lst_col9_15 = [import_total, import_wn, import_bwn, mati_total, mati_wn, mati_bwn]

  lst_cols = lst_col0_8 + lst_col9_15 + lst_col17_20
  lst_df.append(lst_cols)

In [7]:
df = pd.DataFrame(lst_df)
df.insert(0, 'date', tweets_df['date']) 
df['tweet_id'] = tweets_df['tweet_id']
df['link'] = tweets_df['link']
df

,date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,tweet_id,link
0,2021-07-18,"5,778","784,949",85.6,"10,710","916,561","124,593","10,698","8,579","2,119",12,6,6,153,125,28,"7,019",0.77,909,445,1416708430773899268,t.co/fGSlNpO4b4
1,2021-07-19,"6,439","791,388",85.3,"10,972","927,533","128,997","10,924","8,950","1,974",48,28,20,129,118,11,"7,148",0.77,915,435,1417070830656262147,t.co/HSEnf9pkm8


In [8]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [9]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe

sh = gc.open('malaysia covid dataset -- link tableau')
worksheet = sh.worksheet('case data')

def get_next_lastrow(worksheet):
  str_lst = list(filter(None, worksheet.col_values(1)))
  return str(len(str_lst)+1)
  
next_lastrow = int(get_next_lastrow(worksheet))

set_with_dataframe(worksheet, df, row=next_lastrow, col=1, include_index=False, include_column_header=False)

# wget

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

In [14]:
from PIL import Image
from datetime import date

today = str(date.today())    # '2020-06-31'
yyyymmdd = today.split('-')  # ['2021', '06', '31']

dd = yyyymmdd[2]
mm = yyyymmdd[1]
ddmm = '{}{}'.format(yyyymmdd[2],yyyymmdd[1])

# 'http://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/06.06/situasi-terkini-covid-19-di-malaysia-09062021/taburankes-all.jpg'
# 'http://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/06.06/situasi-terkini-covid-19-di-malaysia-08062021/taburankes-all.jpg'
# 'http://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/07.07/situasi-terkini-covid-19-di-malaysia-01072021/taburankes-all.jpg'


url = 'http://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/{}.{}/situasi-terkini-covid-19-di-malaysia-{}2021/taburankes-all.jpg'.format(mm, mm, ddmm)

!wget -P './' {url}    # -O name.jpg

--2021-07-19 12:40:40--  http://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/07.07/situasi-terkini-covid-19-di-malaysia-19072021/taburankes-all.jpg
Resolving covid-19.moh.gov.my (covid-19.moh.gov.my)... 45.124.98.9
Connecting to covid-19.moh.gov.my (covid-19.moh.gov.my)|45.124.98.9|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1359706 (1.3M) [image/jpeg]
Saving to: ‘./taburankes-all.jpg.1’

taburankes-all.jpg. 100%[===================>]   1.30M   255KB/s    in 5.2s    

2021-07-19 12:40:46 (255 KB/s) - ‘./taburankes-all.jpg.1’ saved [1359706/1359706]



In [15]:
try: 
  img = Image.open('./taburankes-all.jpg')
  img.verify()
  img_name = '{}-{}-2021'.format(dd,mm)
  # img_name = text.partition('malaysia-')[2].partition("/taburankes-all")[0]
  !mv "taburankes-all.jpg" {img_name}.jpg
  !cp {img_name}.jpg "/content/drive/MyDrive/covid taburan kes"
  with open("/content/drive/MyDrive/covid taburan kes/taburan kes.txt", "a") as f:
    f.write("{},{}\n".format(today,url))
except (IOError, SyntaxError) as e:
  print('Bad file') 